In [1]:
import pandas as pd
import numpy as np
import ast
import math
import random
import collections
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
import sys
sys.path.append('../../')
from modules.many_features import utils, constants

#### The functions

In [2]:
def streamline_frequency_dict(frequency_dict):
    frequency_dict_list = [] #will hold dictonaries with 2 keys i.e. 'set' and 'value'
    for path_dict, value in frequency_dict.items():
        all_set_list = [i['set'] for i in frequency_dict_list] #get all the sets so far in the list 
        path_set = set(ast.literal_eval(path_dict)) #get the set of current path dict to check if its already in list
        if path_set in all_set_list: #increase value else insert it - work on this!!!!!!!!!!!!!!!
            for elem in frequency_dict_list: #look for if path_set is already in frequency_dict_list 
                if elem['set'] == path_set: #find the matching path_set in the frequency_dict_list
                    elem['value'] += value #increase the value for that set
        else:
            frequency_dict_list.append({'set':path_set, 'value':value})
    
    all_list = [list(path_dict['set']) for path_dict in frequency_dict_list]
    flat_list = [item for sublist in all_list for item in sublist]
    commonest_elements = dict(collections.Counter(flat_list))
    commonest_elements = {k: v for k, v in sorted(commonest_elements.items(), reverse=True, key=lambda item: item[1])}
    commonest_elements_list = list(commonest_elements.keys())
    commonest_elements_list = [i for i in commonest_elements_list if i not in ['Lupus', 'No lupus', 'Inconclusive diagnosis']] + ['Lupus', 'No lupus', 'Inconclusive diagnosis']
    
    for item in frequency_dict_list:
        item['set'] = sorted(list(item['set']), key=lambda x: commonest_elements_list.index(x))
    
    keys = [str(i['set']) for i in frequency_dict_list]
    values = [i['value'] for i in frequency_dict_list]
    final_frequency_dict = {k:v for (k,v) in zip(keys, values)}
    
    return final_frequency_dict

In [3]:
def generate_tuple_dict(df):
    frequency_dict = {}
    for traj in df.trajectory:
        if traj in frequency_dict.keys():
            frequency_dict[traj] += 1
        else:
            frequency_dict[traj] = 1
    streamlined_frequency_dict = streamline_frequency_dict(frequency_dict)
    overall_tup_dict = {}
    for key, value in streamlined_frequency_dict.items():
#     for key, value in frequency_dict.items():
        new_key = ast.literal_eval(key)
        for tup in zip(new_key, new_key[1:]):
            #print(f'tup: {tup}')
            if tup in overall_tup_dict.keys():
                overall_tup_dict[tup] += value
            else:
                overall_tup_dict[tup] = value
    #print(f'overall_tup_dict: {overall_tup_dict}')
    return overall_tup_dict

In [4]:
def create_sankey_df(df):
    overall_tup_dict = generate_tuple_dict(df)
    sankey_df = pd.DataFrame()
    sankey_df['source'] = [i[0] for i in overall_tup_dict.keys()]
    sankey_df['target'] = [i[1] for i in overall_tup_dict.keys()]
    sankey_df['value'] = list(overall_tup_dict.values())
    sankey_df['link_type'] = sankey_df['target'].apply(lambda i: 'terminal' if i in list(constants.CLASS_DICT.keys()) else 'non_terminal')
    return sankey_df

In [5]:
def get_threshold_value(row):
    substring = f"'{row.source}', '{row.target}'"
    substring_df = pd.DataFrame()
    for i, test_row in test_df.iterrows():
        if substring in test_row.trajectory:
            substring_df = substring_df.append(test_row)
    substring_testing_df = testing_df.loc[substring_df.index]
    
    threshold_values = substring_testing_df[row.source].unique().tolist()
    if len(threshold_values) == 1:
        return int(threshold_values[0])
    else:
#         print(f'There is {len(threshold_values)} threshold values for the edge between {row.source} and {row.target}')
        return np.nan

In [6]:
def draw_pyvis_network(pathways_df):
#     pathways_df = create_sankey_df(test_df)
    pathways_df['type'] = 'directed'
    
    got_net = Network(notebook=True, height='750px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value in zip(pathways_df.source, pathways_df.target, pathways_df.value):
        if value > threshold:
            got_net.add_edge(src, target, value=value, color='red')
        else:
            got_net.add_edge(src, target, value=value, color='blue')
    return got_net

In [7]:
def draw_labelled_pyvis_network(pathways_df, pathway_type='network'): #second type is trajectory 
    pathways_df['type'] = 'directed'
    pathways_df['edge_threshold'] = pathways_df.apply(lambda row: get_threshold_value(row), axis=1)
    
    #change get_net to something else since it satnds for Game of Thrones
    got_net = Network(notebook=True, height='750px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value, edge_thresh in zip(pathways_df.source, pathways_df.target, pathways_df.value, pathways_df.edge_threshold):
#         if pathway_type =='network':
        if value > threshold: #this is what mainly changes
            if math.isnan(edge_thresh):
                got_net.add_edge(src, target, value=value, color='red')
            else:
                got_net.add_edge(src, target, value=value, color='red', label=str(int(edge_thresh)))
        else:
            got_net.add_edge(src, target, value=value, color='blue')
#         elif pathway_type =='trajectory': #It's just one trajectory
#             if math.isnan(edge_thresh):
#                 got_net.add_edge(src, target, value=value, color='blue')
#             else:
#                 got_net.add_edge(src, target, value=value, color='blue', label=str(int(edge_thresh)))
#         else:
#             print('Unknown pathway type')
    return got_net

#### The data

In [8]:
testing_df = pd.read_csv('../../final/data/with_new_stable_baselines/testing_set.csv')
testing_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [9]:
# test_df = pd.read_csv('../test_dfs/negative_reward/missingness_0.5.csv').drop(['index'], axis=1)
test_df = pd.read_csv('../../final/test_dfs/new/test_df_basic.csv').drop(['index'], axis=1)
test_df.head()

,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,5.0,1.0,5.0,5.0,"['hemoglobin', 'rbc', 'mcv', 'ret_count', 'Hem...",0.0,1.0
1,5.0,1.0,1.0,1.0,"['hemoglobin', 'rbc', 'mcv', 'segmented_neutro...",0.0,1.0
2,6.0,1.0,4.0,4.0,"['hemoglobin', 'rbc', 'ferritin', 'tibc', 'hem...",0.0,1.0
3,3.0,1.0,0.0,0.0,"['hemoglobin', 'hematocrit', 'No anemia']",0.0,1.0
4,5.0,-1.0,7.0,7.0,"['hemoglobin', 'rbc', 'mcv', 'segmented_neutro...",1.0,1.0


In [10]:
threshold = 0.1*len(test_df)

In [11]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

97.52142857142857

#### The pathways

In [12]:
pathways_df = create_sankey_df(test_df)
pathways_df.head()

,source,target,value,link_type
0,hemoglobin,rbc,12037,non_terminal
1,rbc,mcv,6546,non_terminal
2,mcv,ret_count,3764,non_terminal
3,ret_count,Hemolytic anemia,1835,terminal
4,mcv,segmented_neutrophils,4060,non_terminal


In [13]:
pathways_df.target.unique()

array(['rbc', 'mcv', 'ret_count', 'Hemolytic anemia',
       'segmented_neutrophils', 'Vitamin B12/Folate deficiency anemia',
       'ferritin', 'tibc', 'Iron deficiency anemia', 'hematocrit',
       'No anemia', 'Inconclusive diagnosis', 'gender',
       'Unspecified anemia', 'Anemia of chronic disease',
       'Aplastic anemia', 'tsat', 'creatinine', 'copper', 'glucose'],
      dtype=object)

In [14]:
start_node = pathways_df.iloc[0]['source']
all_nodes = list(set(pathways_df.source.unique().tolist() + pathways_df.target.unique().tolist()))
terminal_nodes = list(set(pathways_df[pathways_df.link_type=='terminal'].target))
non_terminal_nodes = [i for i in all_nodes if i not in terminal_nodes and i!=start_node]

In [15]:
anem_net = draw_pyvis_network(pathways_df)    
anem_net.show('Example.html')

#### Anemia Class 0

In [16]:
test_df_0 = test_df[test_df.y_pred==0]
pathways_df_0 = create_sankey_df(test_df_0)
anem_net_0 = draw_pyvis_network(pathways_df_0)    
anem_net_0.show('Example.html')

#### Labelled network

In [ ]:
labelled_anem_net = draw_labelled_pyvis_network(pathways_df)    
labelled_anem_net.show('Example.html')

#### The Confusion Matrix

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])

#### The Classification Report

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])